In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

file_path_prices = 'DailyPrices.csv'
file_path_portfolio = 'Portfolio.csv'
prices_df = pd.read_csv(file_path_prices)
portfolio_df = pd.read_csv(file_path_portfolio)

In [2]:
# Daily Returns
def return_calculate(prices: pd.DataFrame, method="DISCRETE", date_column="Date"):
    if date_column not in prices.columns:
        raise ValueError(f"date_column: {date_column} not in DataFrame")

    vars = prices.columns.tolist()
    vars.remove(date_column)

    p = prices[vars].to_numpy()
    n, m = p.shape

    p2 = np.zeros((n-1, m))

    if method.upper() == "DISCRETE":
        p2 = (p[1:, :] / p[:-1, :]) - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p[1:, :] / p[:-1, :])
    else:
        raise ValueError(f"Invalid method '{method}'. Must be 'DISCRETE' or 'LOG'.")

    dates = prices[date_column].iloc[1:].reset_index(drop=True)
    returns_df = pd.DataFrame(p2, columns=vars)
    returns_df.insert(0, date_column, dates)
    
    return returns_df


In [3]:
# Exponentially Weighted Covariance Function
def ew_covariance(returns, lambda_=0.97):
    n, m = returns.shape
    weights = np.array([(1 - lambda_) * lambda_ ** i for i in range(n)])[::-1]
    weights /= weights.sum()
    
    mean_returns = np.average(returns, axis=0, weights=weights)
    deviations = returns - mean_returns
    weighted_cov_matrix = np.zeros((m, m))
    
    for i in range(n):
        weighted_cov_matrix += weights[i] * np.outer(deviations[i], deviations[i])
    
    return weighted_cov_matrix

# VaR Calculation
def ewma_var(holdings, covariance_matrix, portfolio_value, alpha=0.05):
    portfolio_weights = holdings / holdings.sum()
    portfolio_variance = portfolio_weights.T @ covariance_matrix @ portfolio_weights
    portfolio_std = np.sqrt(portfolio_variance)
    var_percentage = stats.norm.ppf(alpha) * portfolio_std
    var_dollar = portfolio_value * abs(var_percentage)
    return var_dollar

In [4]:
# Portfolio Value
def calculate_portfolio_value(portfolio_df, latest_prices):
    portfolio_values = {}

    unique_portfolios = portfolio_df['Portfolio'].unique()

    for portfolio in unique_portfolios:
        portfolio_holdings = portfolio_df[portfolio_df['Portfolio'] == portfolio]
        total_value = 0
        for _, row in portfolio_holdings.iterrows():
            stock = row['Stock']
            shares = row['Holding']

            if stock in latest_prices.index:
                stock_price = latest_prices[stock]
                total_value += stock_price * shares

        portfolio_values[portfolio] = total_value
    
    return portfolio_values


In [5]:
arithmetic_returns_df = return_calculate(prices_df, method="DISCRETE", date_column="Date")
log_returns_df = return_calculate(prices_df, method="LOG", date_column="Date")

portfolio_stocks = portfolio_df['Stock'].unique()
available_stocks = [stock for stock in portfolio_stocks if stock in arithmetic_returns_df.columns]

filtered_portfolio_df = portfolio_df[portfolio_df['Stock'].isin(available_stocks)]

filtered_portfolio_A = filtered_portfolio_df[filtered_portfolio_df['Portfolio'] == 'A']['Holding'].values
filtered_portfolio_B = filtered_portfolio_df[filtered_portfolio_df['Portfolio'] == 'B']['Holding'].values
filtered_portfolio_C = filtered_portfolio_df[filtered_portfolio_df['Portfolio'] == 'C']['Holding'].values

# Latest stock prices
latest_prices = prices_df.iloc[-1]

In [6]:
# Calculate portfolio values in $
portfolio_values = calculate_portfolio_value(filtered_portfolio_df, latest_prices)
portfolio_value_A = portfolio_values.get('A', 0)
portfolio_value_B = portfolio_values.get('B', 0)
portfolio_value_C = portfolio_values.get('C', 0)

In [7]:
returns_matrix_A_arithmetic = arithmetic_returns_df[available_stocks].to_numpy()
returns_matrix_A_log = log_returns_df[available_stocks].to_numpy()

returns_matrix_A_arithmetic_filtered = returns_matrix_A_arithmetic[:, :len(filtered_portfolio_A)]
returns_matrix_A_log_filtered = returns_matrix_A_log[:, :len(filtered_portfolio_A)]

returns_matrix_B_arithmetic_filtered = returns_matrix_A_arithmetic[:, :len(filtered_portfolio_B)]
returns_matrix_B_log_filtered = returns_matrix_A_log[:, :len(filtered_portfolio_B)]

returns_matrix_C_arithmetic_filtered = returns_matrix_A_arithmetic[:, :len(filtered_portfolio_C)]
returns_matrix_C_log_filtered = returns_matrix_A_log[:, :len(filtered_portfolio_C)]


In [8]:
cov_matrix_A_arithmetic = ew_covariance(returns_matrix_A_arithmetic_filtered)
cov_matrix_A_log = ew_covariance(returns_matrix_A_log_filtered)

cov_matrix_B_arithmetic = ew_covariance(returns_matrix_B_arithmetic_filtered)
cov_matrix_B_log = ew_covariance(returns_matrix_B_log_filtered)

cov_matrix_C_arithmetic = ew_covariance(returns_matrix_C_arithmetic_filtered)
cov_matrix_C_log = ew_covariance(returns_matrix_C_log_filtered)

In [9]:
var_A_ewma_arithmetic = ewma_var(filtered_portfolio_A, cov_matrix_A_arithmetic, portfolio_value_A)
var_A_ewma_log = ewma_var(filtered_portfolio_A, cov_matrix_A_log, portfolio_value_A)

var_B_ewma_arithmetic = ewma_var(filtered_portfolio_B, cov_matrix_B_arithmetic, portfolio_value_B)
var_B_ewma_log = ewma_var(filtered_portfolio_B, cov_matrix_B_log, portfolio_value_B)

var_C_ewma_arithmetic = ewma_var(filtered_portfolio_C, cov_matrix_C_arithmetic, portfolio_value_C)
var_C_ewma_log = ewma_var(filtered_portfolio_C, cov_matrix_C_log, portfolio_value_C)

In [10]:
total_value = portfolio_value_A + portfolio_value_B + portfolio_value_C
total_holdings = np.concatenate([filtered_portfolio_A, filtered_portfolio_B, filtered_portfolio_C])

total_returns_matrix_arithmetic = np.concatenate([returns_matrix_A_arithmetic_filtered, returns_matrix_B_arithmetic_filtered, returns_matrix_C_arithmetic_filtered], axis=1)
total_returns_matrix_log = np.concatenate([returns_matrix_A_log_filtered, returns_matrix_B_log_filtered, returns_matrix_C_log_filtered], axis=1)

total_cov_matrix_arithmetic = ew_covariance(total_returns_matrix_arithmetic)
total_cov_matrix_log = ew_covariance(total_returns_matrix_log)

total_var_ewma_arithmetic = ewma_var(total_holdings, total_cov_matrix_arithmetic, total_value)
total_var_ewma_log = ewma_var(total_holdings, total_cov_matrix_log, total_value)

In [11]:
print(f"Portfolio A EWMA VaR in $ (Arithmetic Returns): {var_A_ewma_arithmetic}")
print(f"Portfolio A EWMA VaR in $ (Log Returns): {var_A_ewma_log}")

print(f"Portfolio B EWMA VaR in $ (Arithmetic Returns): {var_B_ewma_arithmetic}")
print(f"Portfolio B EWMA VaR in $ (Log Returns): {var_B_ewma_log}")

print(f"Portfolio C EWMA VaR in $ (Arithmetic Returns): {var_C_ewma_arithmetic}")
print(f"Portfolio C EWMA VaR in $ (Log Returns): {var_C_ewma_log}")

print(f"Total Portfolio EWMA VaR in $ (Arithmetic Returns): {total_var_ewma_arithmetic}")
print(f"Total Portfolio EWMA VaR in $ (Log Returns): {total_var_ewma_log}")


Portfolio A EWMA VaR in $ (Arithmetic Returns): 6214.882145710423
Portfolio A EWMA VaR in $ (Log Returns): 6308.772780848762
Portfolio B EWMA VaR in $ (Arithmetic Returns): 6628.232740705449
Portfolio B EWMA VaR in $ (Log Returns): 6665.5789650746865
Portfolio C EWMA VaR in $ (Arithmetic Returns): 4916.82753006697
Portfolio C EWMA VaR in $ (Log Returns): 4931.821147848847
Total Portfolio EWMA VaR in $ (Arithmetic Returns): 17238.033245865045
Total Portfolio EWMA VaR in $ (Log Returns): 17379.775182704838
